# Volumetric Measurement Experiments
## Based on Reference Papers

**References:**
- **박문기** - 체적 측정: Icosphere 해상도별 정밀도, Green's/Divergence Theorem, 비수밀 메쉬 Capping
- **박도현 (DIMA 69차)** - 림프부종 체적 측정: Slice-based Volume, Truncated Cone, Bland-Altman, TEM/CV/ICC

**Experiments:**
1. Icosphere Resolution vs Volume Accuracy (Level 1-5)
2. Green's Theorem 2D - Cross-Section Area
3. Slice-Based Volume (1mm slices + Smoothing Splines + Green's Equation)
4. Truncated Cone Approximation
5. Planar Capping for Non-Watertight Meshes
6. Mesh Resolution Convergence Study
7. Bland-Altman Analysis & Statistical Metrics (TEM, CV, ICC)
8. Comprehensive Comparison & Analysis

---
## 0. Setup

In [ ]:
# --- Environment Setup ---
import os

# Detect environment
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
    PROJECT_DIR = '/content/drive/MyDrive/Volumetric_measurements'
    WORK_DIR = '/content/volumetric_exp'
    os.makedirs(WORK_DIR, exist_ok=True)
    os.chdir(WORK_DIR)
    # Extract bunny data from Drive to local
    import tarfile
    src = os.path.join(PROJECT_DIR, 'bunny.tar.gz')
    if not os.path.exists('bunny'):
        print(f'Extracting {src} ...')
        with tarfile.open(src, 'r:gz') as tar:
            tar.extractall('.')
        print('Done!')
    else:
        print('bunny/ already exists.')
except ImportError:
    IN_COLAB = False
    # Local execution - use current script directory
    WORK_DIR = os.path.dirname(os.path.abspath('volumetric_experiments.ipynb'))
    if not WORK_DIR:
        WORK_DIR = os.getcwd()
    os.chdir(WORK_DIR)
    PROJECT_DIR = WORK_DIR
    print(f'Running locally (not Colab)')
    # Extract if needed
    if not os.path.exists('bunny') and os.path.exists('bunny.tar.gz'):
        import tarfile
        with tarfile.open('bunny.tar.gz', 'r:gz') as tar:
            tar.extractall('.')
        print('Extracted bunny.tar.gz')

print(f'Working directory: {os.getcwd()}')
print(f'Project directory: {PROJECT_DIR}')

In [ ]:
# Install packages (Colab only; local env should have these pre-installed)
if IN_COLAB:
    !pip install -q plotly trimesh rtree scikit-image tqdm shapely
else:
    print('Local environment - skipping pip install')

In [ ]:
import sys, warnings
import numpy as np
import scipy
from scipy.spatial import ConvexHull
from scipy.interpolate import UnivariateSpline, splprep, splev
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import cm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import trimesh
from tqdm import tqdm
from pathlib import Path
import json, shutil

warnings.filterwarnings('ignore')
np.random.seed(42)
print(f'Python: {sys.version}')
print(f'NumPy: {np.__version__}, SciPy: {scipy.__version__}, Trimesh: {trimesh.__version__}')
print('All libraries loaded!')

In [ ]:
# ---- PLY Parser & Data Loading ----
def parse_ply(filepath):
    """Parse ASCII PLY -> vertices (N,D), faces (M,3), property names."""
    with open(filepath, 'r') as f:
        lines = f.readlines()
    n_verts = n_faces = 0
    header_end = 0
    props = []
    in_vertex_elem = False
    for i, line in enumerate(lines):
        s = line.strip()
        if s.startswith('element vertex'):
            n_verts = int(s.split()[-1]); in_vertex_elem = True
        elif s.startswith('element'):
            if 'face' in s: n_faces = int(s.split()[-1])
            in_vertex_elem = False
        elif s.startswith('property') and in_vertex_elem:
            props.append(s.split()[-1])
        elif s == 'end_header':
            header_end = i + 1; break
    verts = np.array([[float(x) for x in lines[header_end + j].split()] for j in range(n_verts)])
    faces = None
    if n_faces > 0:
        face_start = header_end + n_verts
        faces = np.array([[int(x) for x in lines[face_start + j].split()[1:4]] for j in range(n_faces)])
    return verts, faces, props

# Load all bunny resolutions
BASE = Path('bunny/reconstruction')
mesh_files = {
    'Full (35k)': 'bun_zipper.ply',
    'Res2 (8k)': 'bun_zipper_res2.ply',
    'Res3 (1.9k)': 'bun_zipper_res3.ply',
    'Res4 (453)': 'bun_zipper_res4.ply',
}
meshes = {}
for label, fname in mesh_files.items():
    v, f, p = parse_ply(BASE / fname)
    meshes[label] = {'vertices': v[:, :3], 'faces': f}
    print(f'{label:16s} | {v.shape[0]:>6,} verts | {f.shape[0]:>6,} faces')

vertices = meshes['Full (35k)']['vertices']
faces = meshes['Full (35k)']['faces']
vertices_light = meshes['Res2 (8k)']['vertices']
faces_light = meshes['Res2 (8k)']['faces']
print(f'\nMain mesh: {len(vertices):,} verts, {len(faces):,} faces')

---
## Core Volume Functions

In [ ]:
def volume_divergence_theorem(verts, faces):
    """Volume via Divergence Theorem.
    
    V = (1/3) ∮ r·dS = (1/3) Σ_faces [centroid · (cross/2)]
      = (1/6) Σ centroid · cross
    
    where centroid = (v1+v2+v3)/3, cross = (v2-v1)×(v3-v1)
    
    Since centroid·cross = (v1+v2+v3)/3 · cross, we get:
    V = (1/6) Σ (v1+v2+v3)/3 · cross = |Σ contributions| / 6
    """
    v1, v2, v3 = verts[faces[:,0]], verts[faces[:,1]], verts[faces[:,2]]
    cross = np.cross(v2 - v1, v3 - v1)
    centroid = (v1 + v2 + v3) / 3.0
    contributions = np.sum(centroid * cross, axis=1)
    return abs(contributions.sum()) / 6.0

def volume_signed_tetrahedra(verts, faces):
    """V = |sum (1/6) * v1 . (v2 x v3)|"""
    v1, v2, v3 = verts[faces[:,0]], verts[faces[:,1]], verts[faces[:,2]]
    signed_vols = np.sum(v1 * np.cross(v2, v3), axis=1) / 6.0
    return abs(signed_vols.sum())

def greens_theorem_2d(x, y):
    """Area of 2D closed polygon using Green's theorem: A = 0.5 * |sum(x_i*y_{i+1} - x_{i+1}*y_i)|
    Also known as the Shoelace formula, derived from Green's theorem."""
    x = np.asarray(x)
    y = np.asarray(y)
    n = len(x)
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += x[i] * y[j] - x[j] * y[i]
    return abs(area) / 2.0

def greens_theorem_2d_vectorized(x, y):
    """Vectorized Green's theorem (Shoelace formula)."""
    x, y = np.asarray(x), np.asarray(y)
    return abs(np.sum(x * np.roll(y, -1) - np.roll(x, -1) * y)) / 2.0

print('Core volume functions defined.')

---
## Experiment 1: Icosphere Resolution vs Volume Accuracy

**Reference: 박문기 p.2**

Generate icospheres (unit sphere approximations) at Level 1-5 subdivision.  
Compare computed volume (Divergence Theorem) with analytical volume $V = \frac{4}{3}\pi r^3 = 4.18879...$

| Level | Vertices | Faces | Expected Error |
|-------|----------|-------|-----------|
| 1     | 42       | 80    | ~12.65%   |
| 2     | 162      | 320   | ~3.37%    |
| 3     | 642      | 1280  | ~0.856%   |
| 4     | 2562     | 5120  | ~0.215%   |
| 5     | 10242    | 20480 | ~0.054%   |

In [ ]:
def create_icosphere(subdivisions=0, radius=1.0):
    """Create an icosphere by subdividing an icosahedron."""
    # Base icosahedron
    t = (1.0 + np.sqrt(5.0)) / 2.0
    verts = np.array([
        [-1, t, 0], [1, t, 0], [-1, -t, 0], [1, -t, 0],
        [0, -1, t], [0, 1, t], [0, -1, -t], [0, 1, -t],
        [t, 0, -1], [t, 0, 1], [-t, 0, -1], [-t, 0, 1],
    ], dtype=np.float64)
    # Normalize to unit sphere
    norms = np.linalg.norm(verts, axis=1, keepdims=True)
    verts = verts / norms

    faces = np.array([
        [0,11,5],[0,5,1],[0,1,7],[0,7,10],[0,10,11],
        [1,5,9],[5,11,4],[11,10,2],[10,7,6],[7,1,8],
        [3,9,4],[3,4,2],[3,2,6],[3,6,8],[3,8,9],
        [4,9,5],[2,4,11],[6,2,10],[8,6,7],[9,8,1],
    ], dtype=np.int64)

    # Subdivide
    for _ in range(subdivisions):
        edge_midpoints = {}
        new_faces = []
        n_verts = len(verts)

        def get_midpoint(i, j):
            key = (min(i, j), max(i, j))
            if key in edge_midpoints:
                return edge_midpoints[key]
            mid = (verts[i] + verts[j]) / 2.0
            mid = mid / np.linalg.norm(mid)  # Project to sphere
            idx = len(verts_list)
            verts_list.append(mid)
            edge_midpoints[key] = idx
            return idx

        verts_list = list(verts)
        for tri in faces:
            a, b, c = tri
            ab = get_midpoint(a, b)
            bc = get_midpoint(b, c)
            ca = get_midpoint(c, a)
            new_faces.extend([[a, ab, ca], [b, bc, ab], [c, ca, bc], [ab, bc, ca]])

        verts = np.array(verts_list)
        faces = np.array(new_faces, dtype=np.int64)

    verts = verts * radius
    return verts, faces

# Analytical volume of unit sphere
V_analytical = (4.0 / 3.0) * np.pi  # = 4.188790...
A_analytical = 4.0 * np.pi            # = 12.566371...

print(f'Analytical sphere volume: {V_analytical:.6f}')
print(f'Analytical sphere area  : {A_analytical:.6f}')
print()

ico_results = []
for level in range(1, 6):
    v, f = create_icosphere(subdivisions=level, radius=1.0)
    vol_div = volume_divergence_theorem(v, f)
    vol_tet = volume_signed_tetrahedra(v, f)
    mesh_tm = trimesh.Trimesh(vertices=v, faces=f)
    area = mesh_tm.area
    err_vol = abs(vol_div - V_analytical) / V_analytical * 100
    err_area = abs(area - A_analytical) / A_analytical * 100

    ico_results.append({
        'Level': level,
        'Vertices': len(v),
        'Faces': len(f),
        'Volume (Div)': vol_div,
        'Volume (Tet)': vol_tet,
        'Surface Area': area,
        'Vol Error (%)': err_vol,
        'Area Error (%)': err_area,
        'Watertight': mesh_tm.is_watertight,
    })
    print(f'Level {level}: {len(v):>6,} verts, {len(f):>6,} faces | '
          f'V={vol_div:.6f} (err={err_vol:.4f}%) | A={area:.6f} (err={err_area:.4f}%)')

df_ico = pd.DataFrame(ico_results)
print(f'\nTarget Volume: {V_analytical:.6f}')
print(f'Level 5 Error: {df_ico.iloc[-1]["Vol Error (%)"]:.4f}%')

In [ ]:
# Visualization: Icosphere levels + convergence plot
fig = make_subplots(
    rows=1, cols=5,
    specs=[[{'type':'scene'}]*5],
    subplot_titles=[f'Level {i} ({df_ico.iloc[i-1]["Vertices"]} verts)' for i in range(1,6)],
)
for level in range(1, 6):
    v, f = create_icosphere(subdivisions=level, radius=1.0)
    fig.add_trace(go.Mesh3d(
        x=v[:,0], y=v[:,1], z=v[:,2],
        i=f[:,0], j=f[:,1], k=f[:,2],
        color='steelblue', opacity=0.8, flatshading=True, showscale=False,
    ), row=1, col=level)

for i in range(1, 6):
    sc = 'scene' if i == 1 else f'scene{i}'
    fig.update_layout(**{sc: dict(aspectmode='data', camera=dict(eye=dict(x=1.5,y=1.5,z=1.5)))})
fig.update_layout(title='Icosphere Subdivision Levels 1-5', width=1500, height=350)
fig.show()

# Convergence plot
fig2, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.semilogy(df_ico['Level'], df_ico['Vol Error (%)'], 'bo-', markersize=10, linewidth=2, label='Volume Error')
ax1.semilogy(df_ico['Level'], df_ico['Area Error (%)'], 'rs-', markersize=10, linewidth=2, label='Area Error')
ax1.set_xlabel('Icosphere Level', fontsize=12)
ax1.set_ylabel('Relative Error (%)', fontsize=12)
ax1.set_title('Icosphere: Error vs Subdivision Level (박문기 Table 재현)', fontsize=13, fontweight='bold')
ax1.legend(fontsize=11); ax1.grid(True, alpha=0.3)
ax1.set_xticks(range(1, 6))

ax2.semilogy(df_ico['Vertices'], df_ico['Vol Error (%)'], 'bo-', markersize=10, linewidth=2, label='Volume Error')
ax2.semilogy(df_ico['Vertices'], df_ico['Area Error (%)'], 'rs-', markersize=10, linewidth=2, label='Area Error')
ax2.set_xlabel('Number of Vertices', fontsize=12)
ax2.set_ylabel('Relative Error (%)', fontsize=12)
ax2.set_title('Error vs Mesh Resolution', fontsize=13, fontweight='bold')
ax2.legend(fontsize=11); ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print table matching reference paper format
print('\n=== Icosphere Precision Table (Reference: 박문기 p.2) ===')
print(f'{"Level":>5} {"Vertices":>10} {"Faces":>8} {"Volume":>12} {"Error (%)":>12} {"Area":>12} {"Area Err(%)":>12}')
print('-' * 80)
for _, r in df_ico.iterrows():
    print(f'{int(r["Level"]):>5} {int(r["Vertices"]):>10,} {int(r["Faces"]):>8,} '
          f'{r["Volume (Div)"]:>12.6f} {r["Vol Error (%)"]:>12.4f} '
          f'{r["Surface Area"]:>12.6f} {r["Area Error (%)"]:>12.4f}')
print(f'{"Exact":>5} {"-":>10} {"-":>8} {V_analytical:>12.6f} {0.0:>12.4f} {A_analytical:>12.6f} {0.0:>12.4f}')

---
## Experiment 2: Green's Theorem 2D - Cross-Section Area

**Reference: 박문기 (Green's Theorem), 박도현 (Green's Equations per slice)**

Green's Theorem converts a double integral over a region to a line integral along its boundary:

$$A = \oint_C x\,dy = \frac{1}{2} \oint_C (x\,dy - y\,dx) = \frac{1}{2}\sum_{i=0}^{n-1}(x_i y_{i+1} - x_{i+1} y_i)$$

This is the **Shoelace formula**, directly derived from Green's theorem.  
We apply this to cross-sections of the Stanford Bunny mesh.

In [ ]:
def mesh_cross_section(verts, faces, axis=1, level=None, tolerance=None):
    """Extract cross-section contour of a mesh at a given axis level.
    Returns ordered 2D points of the cross-section boundary."""
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    if level is None:
        level = verts[:, axis].mean()
    
    # Use trimesh section
    origin = np.zeros(3)
    origin[axis] = level
    normal = np.zeros(3)
    normal[axis] = 1.0
    
    section = mesh.section(plane_origin=origin, plane_normal=normal)
    if section is None:
        return None, None
    
    # Get 2D path
    try:
        path_2d, transform = section.to_planar()
        # Get vertices from the path
        if len(path_2d.polygons_closed) > 0:
            # Get the largest polygon
            polygon = max(path_2d.polygons_closed, key=lambda p: p.area)
            coords = np.array(polygon.exterior.coords)
            return coords[:, 0], coords[:, 1]
    except Exception:
        pass
    return None, None


# Demonstrate Green's theorem on unit circle first (validation)
print('=== Validation: Green\'s Theorem on Unit Circle ===')
for n_pts in [10, 50, 100, 500, 1000]:
    theta = np.linspace(0, 2*np.pi, n_pts, endpoint=False)
    cx, cy = np.cos(theta), np.sin(theta)
    area = greens_theorem_2d_vectorized(cx, cy)
    err = abs(area - np.pi) / np.pi * 100
    print(f'  n={n_pts:>5}: Area={area:.8f}, Error={err:.6f}% (exact={np.pi:.8f})')

# Cross-sections of Stanford Bunny
print('\n=== Stanford Bunny Cross-Sections (Green\'s Theorem) ===')
y_min, y_max = vertices[:, 1].min(), vertices[:, 1].max()
y_range = y_max - y_min
n_slices_demo = 10
y_levels = np.linspace(y_min + y_range * 0.1, y_max - y_range * 0.1, n_slices_demo)

cross_sections = []
for y_lev in y_levels:
    cx, cy = mesh_cross_section(vertices, faces, axis=1, level=y_lev)
    if cx is not None:
        area = greens_theorem_2d_vectorized(cx, cy)
        cross_sections.append({'y_level': y_lev, 'area': area, 'n_points': len(cx), 'cx': cx, 'cy': cy})
        print(f'  y={y_lev:+.5f}: {len(cx):>4} pts, Area={area:.8f}')
    else:
        print(f'  y={y_lev:+.5f}: No cross-section found')

print(f'\nSuccessful cross-sections: {len(cross_sections)}/{n_slices_demo}')

In [ ]:
# Visualize cross-sections
n_show = min(len(cross_sections), 6)
if n_show > 0:
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    axes = axes.ravel()
    step = max(1, len(cross_sections) // n_show)
    shown = 0
    for idx in range(0, len(cross_sections), step):
        if shown >= n_show:
            break
        cs = cross_sections[idx]
        ax = axes[shown]
        ax.fill(cs['cx'], cs['cy'], alpha=0.3, color='steelblue')
        ax.plot(cs['cx'], cs['cy'], 'b-', linewidth=1)
        ax.set_title(f"y={cs['y_level']:.4f}\nArea={cs['area']:.6f} ({cs['n_points']} pts)", fontsize=10)
        ax.set_aspect('equal')
        ax.grid(True, alpha=0.3)
        shown += 1
    for i in range(shown, len(axes)):
        axes[i].set_visible(False)
    plt.suptitle('Stanford Bunny - Cross-Sections (Green\'s Theorem Area)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print('No cross-sections available for visualization.')

---
## Experiment 3: Slice-Based Volume Measurement

**Reference: 박도현 p.7 - Smoothing Splines + Green's Equations**

Algorithm:
1. Slice mesh along Y-axis at regular intervals (1mm equivalent)
2. Extract boundary contour at each slice
3. Apply **smoothing splines** to smooth the contour
4. Compute area of each slice using **Green's theorem (Shoelace formula)**
5. Integrate areas using **trapezoidal rule** to get total volume

$$V_{\text{slice}} = \int_{y_{\min}}^{y_{\max}} A(y)\,dy \approx \sum_{i=0}^{n-1} \frac{A(y_i) + A(y_{i+1})}{2} \cdot \Delta y$$

In [ ]:
def smooth_contour(cx, cy, smoothing_factor=0.0, n_resample=200):
    """Smooth a 2D contour using parametric smoothing splines.
    Reference: smoothing splines on cross-section contours."""
    if not (np.allclose(cx[0], cx[-1]) and np.allclose(cy[0], cy[-1])):
        cx = np.append(cx, cx[0])
        cy = np.append(cy, cy[0])
    try:
        tck, u = splprep([cx, cy], s=smoothing_factor, per=True, k=3)
        u_new = np.linspace(0, 1, n_resample)
        cx_smooth, cy_smooth = splev(u_new, tck)
        return cx_smooth, cy_smooth
    except Exception:
        return cx, cy


def slice_based_volume(verts, faces, axis=1, n_slices=100, smooth=True, smoothing_factor=0.0):
    """Compute volume by slicing mesh and integrating cross-section areas.
    Reference: 1mm slices + smoothing splines + Green's equations."""
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    lo, hi = verts[:, axis].min(), verts[:, axis].max()
    margin = (hi - lo) * 0.02
    levels = np.linspace(lo + margin, hi - margin, n_slices)
    
    areas = []
    valid_levels = []
    
    for lev in levels:
        origin = np.zeros(3); origin[axis] = lev
        normal = np.zeros(3); normal[axis] = 1.0
        section = mesh.section(plane_origin=origin, plane_normal=normal)
        if section is None:
            continue
        try:
            path_2d, _ = section.to_planar()
            if len(path_2d.polygons_closed) > 0:
                polygon = max(path_2d.polygons_closed, key=lambda p: p.area)
                coords = np.array(polygon.exterior.coords)
                cx, cy = coords[:, 0], coords[:, 1]
                
                if smooth and len(cx) >= 4:
                    cx, cy = smooth_contour(cx, cy, smoothing_factor=smoothing_factor)
                
                area = greens_theorem_2d_vectorized(cx, cy)
                areas.append(area)
                valid_levels.append(lev)
        except Exception:
            continue
    
    if len(areas) < 2:
        return 0.0, np.array([]), np.array([])
    
    areas = np.array(areas)
    valid_levels = np.array(valid_levels)
    
    # Trapezoidal integration (np.trapezoid for NumPy 2.x)
    _trapz = getattr(np, 'trapezoid', getattr(np, 'trapz', None))
    volume = _trapz(areas, valid_levels)
    return volume, valid_levels, areas


# Run slice-based volume with different slice counts and smoothing
print('=== Slice-Based Volume Measurement ===')
print('(Reference: Smoothing Splines + Green\'s Equations)\n')

slice_results = []
vol_ref = volume_divergence_theorem(vertices, faces)

for n_sl in [20, 50, 100, 200]:
    for smooth, s_factor, s_label in [(False, 0.0, 'None'), (True, 0.0, 's=0'), (True, 1e-6, 's=1e-6')]:
        vol, levels, areas = slice_based_volume(vertices, faces, n_slices=n_sl, 
                                                smooth=smooth, smoothing_factor=s_factor)
        err = abs(vol - vol_ref) / vol_ref * 100 if vol_ref > 0 else float('inf')
        slice_results.append({
            'Slices': n_sl,
            'Smoothing': s_label,
            'Volume': vol,
            'Valid Slices': len(areas),
            'Err vs DivThm (%)': err,
        })
        print(f'  n={n_sl:>3}, smooth={s_label:>6}: V={vol:.8f} ({len(areas)} valid slices, err={err:.2f}%)')

df_slice = pd.DataFrame(slice_results)
print(f'\nDivergence Theorem reference: {vol_ref:.8f}')

In [ ]:
# Detailed visualization for best slice-based result
vol_best, levels_best, areas_best = slice_based_volume(vertices, faces, n_slices=100, 
                                                        smooth=True, smoothing_factor=0.0)

if len(areas_best) > 1:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Area profile
    ax1.fill_between(levels_best, areas_best, alpha=0.3, color='steelblue')
    ax1.plot(levels_best, areas_best, 'b-', linewidth=2)
    ax1.set_xlabel('Y-axis Position', fontsize=12)
    ax1.set_ylabel('Cross-Section Area', fontsize=12)
    ax1.set_title(f'Slice Area Profile (V={vol_best:.6f})', fontsize=13, fontweight='bold')
    ax1.grid(True, alpha=0.3)

    # Cumulative volume
    dy = np.diff(levels_best)
    avg_areas = (areas_best[:-1] + areas_best[1:]) / 2
    cum_vol = np.cumsum(avg_areas * dy)
    ax2.plot(levels_best[1:], cum_vol, 'r-', linewidth=2)
    ax2.axhline(y=vol_best, color='gray', linestyle='--', alpha=0.5, label=f'Total={vol_best:.6f}')
    ax2.set_xlabel('Y-axis Position', fontsize=12)
    ax2.set_ylabel('Cumulative Volume', fontsize=12)
    ax2.set_title('Cumulative Volume Along Slicing Axis', fontsize=13, fontweight='bold')
    ax2.legend(fontsize=11); ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()
else:
    print('No valid slices - skipping visualization')

# Show smoothing effect
print('\n=== Smoothing Effect on Slice-Based Volume ===')
df_smooth = df_slice.pivot_table(index='Slices', columns='Smoothing', values='Volume')
print(df_smooth.to_string())

---
## Experiment 4: Truncated Cone Approximation

**Reference: 박도현 p.5 - LiDAR volume measurement formula**

Approximate volume as a stack of truncated cones (frustums) between adjacent slices:

$$V = \frac{1}{3} \pi h (R^2 + Rr + r^2)$$

where $h$ = slice spacing, $R$ and $r$ are equivalent radii from circumference at each slice:
$$r = \frac{C}{2\pi} = \sqrt{\frac{A}{\pi}}$$

In [ ]:
def volume_truncated_cone(verts, faces, axis=1, n_slices=100):
    """Volume via truncated cone (frustum) approximation.
    Reference: 박도현 p.5 - V = (1/3)*pi*h*(R^2 + R*r + r^2)"""
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    lo, hi = verts[:, axis].min(), verts[:, axis].max()
    margin = (hi - lo) * 0.02
    levels = np.linspace(lo + margin, hi - margin, n_slices)
    
    radii = []
    valid_levels = []
    
    for lev in levels:
        origin = np.zeros(3); origin[axis] = lev
        normal = np.zeros(3); normal[axis] = 1.0
        section = mesh.section(plane_origin=origin, plane_normal=normal)
        if section is None:
            continue
        try:
            path_2d, _ = section.to_planar()
            if len(path_2d.polygons_closed) > 0:
                polygon = max(path_2d.polygons_closed, key=lambda p: p.area)
                area = polygon.area
                # Equivalent radius: r = sqrt(A/pi)
                r_eq = np.sqrt(area / np.pi)
                radii.append(r_eq)
                valid_levels.append(lev)
        except Exception:
            continue
    
    if len(radii) < 2:
        return 0.0, np.array([]), np.array([])
    
    radii = np.array(radii)
    valid_levels = np.array(valid_levels)
    
    # Compute volume as sum of truncated cones
    total_vol = 0.0
    for i in range(len(radii) - 1):
        h = valid_levels[i+1] - valid_levels[i]
        R, r = radii[i], radii[i+1]
        vol_cone = (1.0/3.0) * np.pi * h * (R**2 + R*r + r**2)
        total_vol += vol_cone
    
    return total_vol, valid_levels, radii


# Validate on unit sphere first
print('=== Validation: Truncated Cone on Unit Sphere ===')
v_sphere, f_sphere = create_icosphere(subdivisions=4, radius=1.0)
for n_sl in [20, 50, 100, 200]:
    vol_tc, _, _ = volume_truncated_cone(v_sphere, f_sphere, axis=1, n_slices=n_sl)
    err = abs(vol_tc - V_analytical) / V_analytical * 100
    print(f'  n={n_sl:>3}: V={vol_tc:.6f} (err={err:.4f}%, exact={V_analytical:.6f})')

# Apply to Stanford Bunny
print('\n=== Stanford Bunny: Truncated Cone Approximation ===')
tc_results = []
for n_sl in [20, 50, 100, 200]:
    vol_tc, tc_levels, tc_radii = volume_truncated_cone(vertices, faces, n_slices=n_sl)
    err_vs_div = abs(vol_tc - vol_ref) / vol_ref * 100 if vol_ref > 0 else float('inf')
    tc_results.append({'Slices': n_sl, 'Volume': vol_tc, 'Valid': len(tc_radii), 'Err vs DivThm (%)': err_vs_div})
    print(f'  n={n_sl:>3}: V={vol_tc:.8f} ({len(tc_radii)} valid, err={err_vs_div:.2f}%)')

df_tc = pd.DataFrame(tc_results)

# Visualize radius profile
vol_tc_best, tc_levels_best, tc_radii_best = volume_truncated_cone(vertices, faces, n_slices=100)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(tc_levels_best, tc_radii_best, 'g-o', markersize=3, linewidth=1.5)
ax1.set_xlabel('Y-axis Position', fontsize=12)
ax1.set_ylabel('Equivalent Radius', fontsize=12)
ax1.set_title('Equivalent Radius Profile (r = sqrt(A/pi))', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Compare slice methods
methods = ['Slice (no smooth)', 'Slice (s=0)', 'Slice (s=1e-6)', 'Truncated Cone']
# Get best results for each
slice_100_ns = df_slice[(df_slice['Slices']==100) & (df_slice['Smoothing']=='None')]['Volume'].values[0]
slice_100_s0 = df_slice[(df_slice['Slices']==100) & (df_slice['Smoothing']=='s=0')]['Volume'].values[0]
slice_100_s6 = df_slice[(df_slice['Slices']==100) & (df_slice['Smoothing']=='s=1e-6')]['Volume'].values[0]
tc_100 = df_tc[df_tc['Slices']==100]['Volume'].values[0]

vols_compare = [slice_100_ns, slice_100_s0, slice_100_s6, tc_100]
colors_c = ['#2196F3', '#1976D2', '#0D47A1', '#4CAF50']
ax2.bar(methods, vols_compare, color=colors_c, edgecolor='black', linewidth=0.5)
ax2.axhline(y=vol_ref, color='red', linestyle='--', alpha=0.6, label=f'Div.Thm={vol_ref:.6f}')
ax2.set_ylabel('Volume', fontsize=12)
ax2.set_title('Slice-Based Methods Comparison (100 slices)', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10); ax2.tick_params(axis='x', rotation=15)

plt.tight_layout()
plt.show()

---
## Experiment 5: Planar Capping for Non-Watertight Meshes

**Reference: 박문기 p.4 - Capping (DeepSDF/Planar)**

The Stanford Bunny mesh is **not watertight** (has boundary edges / holes).  
This significantly affects volume accuracy.

**Planar Capping** strategy:
1. Detect boundary edges (edges belonging to only one face)
2. Group boundary edges into loops
3. Fill each loop with triangles (ear-clipping or fan triangulation)
4. Compare volume before/after capping

In [ ]:
def find_boundary_edges(faces):
    """Find boundary edges (edges used by only one face)."""
    edge_count = {}
    for face in faces:
        for i in range(3):
            e = tuple(sorted([face[i], face[(i+1) % 3]]))
            edge_count[e] = edge_count.get(e, 0) + 1
    boundary = [e for e, c in edge_count.items() if c == 1]
    return boundary


def order_boundary_loop(boundary_edges):
    """Order boundary edges into connected loops."""
    if not boundary_edges:
        return []
    
    adj = {}
    for e in boundary_edges:
        adj.setdefault(e[0], []).append(e[1])
        adj.setdefault(e[1], []).append(e[0])
    
    visited_edges = set()
    loops = []
    
    for start_edge in boundary_edges:
        start = start_edge[0]
        if start in visited_edges:
            continue
        
        loop = [start]
        visited_edges.add(start)
        current = start
        
        while True:
            found_next = False
            for neighbor in adj.get(current, []):
                if neighbor not in visited_edges:
                    loop.append(neighbor)
                    visited_edges.add(neighbor)
                    current = neighbor
                    found_next = True
                    break
            if not found_next:
                break
        
        if len(loop) >= 3:
            loops.append(loop)
    
    return loops


def cap_mesh(verts, faces):
    """Cap holes in mesh using fan triangulation from boundary loop centroid."""
    boundary_edges = find_boundary_edges(faces)
    if not boundary_edges:
        return verts, faces, 0
    
    loops = order_boundary_loop(boundary_edges)
    new_verts = list(verts)
    new_faces = list(faces)
    n_cap_faces = 0
    
    for loop in loops:
        if len(loop) < 3:
            continue
        # Add centroid vertex
        centroid = verts[loop].mean(axis=0)
        centroid_idx = len(new_verts)
        new_verts.append(centroid)
        
        # Fan triangulation from centroid
        for i in range(len(loop)):
            j = (i + 1) % len(loop)
            new_faces.append([loop[i], loop[j], centroid_idx])
            n_cap_faces += 1
    
    return np.array(new_verts), np.array(new_faces), n_cap_faces


# Analyze boundary of all bunny meshes
print('=== Boundary Analysis ===')
for label, data in meshes.items():
    v, f = data['vertices'], data['faces']
    boundary = find_boundary_edges(f)
    loops = order_boundary_loop(boundary)
    mesh_tm = trimesh.Trimesh(vertices=v, faces=f)
    print(f'{label:16s}: {len(boundary):>4} boundary edges, '
          f'{len(loops)} loops, watertight={mesh_tm.is_watertight}')

# Cap the full mesh
print('\n=== Capping Stanford Bunny (Full 35k) ===')
v_capped, f_capped, n_cap = cap_mesh(vertices, faces)
print(f'Original : {len(vertices):,} verts, {len(faces):,} faces')
print(f'Capped   : {len(v_capped):,} verts, {len(f_capped):,} faces (+{n_cap} cap faces)')

# Compare volumes
mesh_orig = trimesh.Trimesh(vertices=vertices, faces=faces)
mesh_capped = trimesh.Trimesh(vertices=v_capped, faces=f_capped)

vol_orig_div = volume_divergence_theorem(vertices, faces)
vol_orig_tet = volume_signed_tetrahedra(vertices, faces)
vol_capped_div = volume_divergence_theorem(v_capped, f_capped)
vol_capped_tet = volume_signed_tetrahedra(v_capped, f_capped)

print(f'\n{"Method":>25} {"Original":>14} {"Capped":>14} {"Change":>10}')
print('-' * 65)
print(f'{"Divergence Theorem":>25} {vol_orig_div:>14.8f} {vol_capped_div:>14.8f} '
      f'{(vol_capped_div/vol_orig_div - 1)*100:>+9.2f}%')
print(f'{"Signed Tetrahedra":>25} {vol_orig_tet:>14.8f} {vol_capped_tet:>14.8f} '
      f'{(vol_capped_tet/vol_orig_tet - 1)*100:>+9.2f}%')
print(f'{"Trimesh":>25} {abs(mesh_orig.volume):>14.8f} {abs(mesh_capped.volume):>14.8f} '
      f'{(abs(mesh_capped.volume)/abs(mesh_orig.volume) - 1)*100:>+9.2f}%')
print(f'{"Watertight?":>25} {str(mesh_orig.is_watertight):>14} {str(mesh_capped.is_watertight):>14}')

# Also try trimesh's built-in fill_holes
mesh_filled = mesh_orig.copy()
trimesh.repair.fill_holes(mesh_filled)
trimesh.repair.fix_winding(mesh_filled)
vol_filled = abs(mesh_filled.volume)
print(f'{"Trimesh fill_holes":>25} {"-":>14} {vol_filled:>14.8f} '
      f'{(vol_filled/abs(mesh_orig.volume) - 1)*100:>+9.2f}%')
print(f'{"  Watertight?":>25} {"-":>14} {str(mesh_filled.is_watertight):>14}')

In [ ]:
# Visualize boundary edges and capping
boundary_edges = find_boundary_edges(faces)
boundary_verts = list(set([v for e in boundary_edges for v in e]))
bv = vertices[boundary_verts]

fig = go.Figure()
fig.add_trace(go.Mesh3d(
    x=vertices[:,0], y=vertices[:,2], z=vertices[:,1],
    i=faces[:,0], j=faces[:,1], k=faces[:,2],
    opacity=0.3, color='lightblue', flatshading=True, name='Mesh',
))
fig.add_trace(go.Scatter3d(
    x=bv[:,0], y=bv[:,2], z=bv[:,1],
    mode='markers',
    marker=dict(size=3, color='red'),
    name=f'Boundary ({len(boundary_verts)} verts)',
))
fig.update_layout(
    title=f'Non-Watertight Mesh: Boundary Edges ({len(boundary_edges)} edges)',
    scene=dict(aspectmode='data'),
    width=900, height=700,
)
fig.show()

---
## Experiment 6: Mesh Resolution Convergence Study

**Reference: 박문기 p.2,4 - Point count vs error rate**

Two studies:
1. **Icosphere convergence**: Analytical ground truth available (sphere)
2. **Bunny resolution convergence**: Compare 4 resolution levels (Res4 → Full)

In [ ]:
# Bunny resolution convergence
print('=== Bunny Mesh Resolution Convergence ===')
print(f'{"Resolution":>16} {"Vertices":>10} {"Faces":>8} {"DivThm Vol":>14} {"Tet Vol":>14} {"Area":>12} {"WT":>5}')
print('-' * 80)

res_data = []
for label in ['Res4 (453)', 'Res3 (1.9k)', 'Res2 (8k)', 'Full (35k)']:
    v, f = meshes[label]['vertices'], meshes[label]['faces']
    vd = volume_divergence_theorem(v, f)
    vt = volume_signed_tetrahedra(v, f)
    mt = trimesh.Trimesh(vertices=v, faces=f)
    res_data.append({
        'Label': label, 'Vertices': len(v), 'Faces': len(f),
        'Vol_Div': vd, 'Vol_Tet': vt, 'Area': mt.area,
        'Watertight': mt.is_watertight, 'Euler': mt.euler_number,
    })
    print(f'{label:>16} {len(v):>10,} {len(f):>8,} {vd:>14.8f} {vt:>14.8f} {mt.area:>12.8f} {str(mt.is_watertight):>5}')

df_res = pd.DataFrame(res_data)

# Use Full mesh as reference for error calculation
ref_vol = df_res.iloc[-1]['Vol_Div']
df_res['Err_vs_Full (%)'] = abs(df_res['Vol_Div'] - ref_vol) / ref_vol * 100

print(f'\nReference (Full): {ref_vol:.8f}')
for _, r in df_res.iterrows():
    print(f'  {r["Label"]:>16}: {r["Err_vs_Full (%)"]:.4f}% error')

# Also cap each resolution and compare
print('\n=== After Capping ===')
capped_data = []
for label in ['Res4 (453)', 'Res3 (1.9k)', 'Res2 (8k)', 'Full (35k)']:
    v, f = meshes[label]['vertices'], meshes[label]['faces']
    vc, fc, nc = cap_mesh(v, f)
    vd = volume_divergence_theorem(vc, fc)
    vt = volume_signed_tetrahedra(vc, fc)
    mt = trimesh.Trimesh(vertices=vc, faces=fc)
    capped_data.append({
        'Label': label, 'Cap Faces': nc,
        'Vol_Div': vd, 'Vol_Tet': vt, 'Watertight': mt.is_watertight,
    })
    print(f'  {label:>16} +{nc:>3} caps: DivThm={vd:.8f}, Tet={vt:.8f}, WT={mt.is_watertight}')

df_capped = pd.DataFrame(capped_data)

In [ ]:
# Convergence visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Icosphere convergence (log-log)
ax = axes[0]
ax.loglog(df_ico['Vertices'], df_ico['Vol Error (%)'], 'bo-', markersize=10, linewidth=2, label='Volume')
ax.loglog(df_ico['Vertices'], df_ico['Area Error (%)'], 'rs-', markersize=10, linewidth=2, label='Area')
# Fit power law
log_v = np.log(df_ico['Vertices'].values)
log_e = np.log(df_ico['Vol Error (%)'].values)
slope, intercept = np.polyfit(log_v, log_e, 1)
ax.set_xlabel('Vertices (log)', fontsize=12)
ax.set_ylabel('Error % (log)', fontsize=12)
ax.set_title(f'Icosphere Convergence\n(slope={slope:.2f})', fontsize=13, fontweight='bold')
ax.legend(fontsize=10); ax.grid(True, alpha=0.3, which='both')

# 2. Bunny resolution convergence
ax = axes[1]
ax.plot(df_res['Vertices'], df_res['Vol_Div'], 'bo-', markersize=10, linewidth=2, label='Div. Theorem')
ax.plot(df_res['Vertices'], df_res['Vol_Tet'], 'rs--', markersize=10, linewidth=2, label='Signed Tet.')
ax.set_xlabel('Vertices', fontsize=12)
ax.set_ylabel('Volume', fontsize=12)
ax.set_title('Bunny: Volume vs Resolution', fontsize=13, fontweight='bold')
ax.legend(fontsize=10); ax.grid(True, alpha=0.3)

# 3. Bunny before/after capping
ax = axes[2]
x_pos = np.arange(len(df_res))
w = 0.35
ax.bar(x_pos - w/2, df_res['Vol_Div'], w, color='steelblue', label='Original', edgecolor='black', linewidth=0.5)
ax.bar(x_pos + w/2, df_capped['Vol_Div'], w, color='coral', label='Capped', edgecolor='black', linewidth=0.5)
ax.set_xticks(x_pos)
ax.set_xticklabels([l.split()[0] for l in df_res['Label']], fontsize=9)
ax.set_ylabel('Volume (Div. Theorem)', fontsize=12)
ax.set_title('Volume Before/After Capping', fontsize=13, fontweight='bold')
ax.legend(fontsize=10); ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---
## Experiment 7: Bland-Altman Analysis & Statistical Metrics

**Reference: 박도현 p.5,7,8**

### Bland-Altman Analysis
Compare agreement between measurement methods:
- Mean difference (bias)
- 95% limits of agreement (mean diff +/- 1.96*SD)

### Statistical Metrics
- **TEM** (Technical Error of Measurement): $\text{TEM} = \sqrt{\frac{\sum d_i^2}{2n}}$
- **CV** (Coefficient of Variation): $\text{CV} = \frac{\text{SD}}{\bar{x}} \times 100$
- **ICC** (Intraclass Correlation Coefficient): consistency across methods

In [ ]:
# Collect all volume measurements across resolutions and methods
print('=== Collecting Volume Measurements Across Resolutions ===')

all_measurements = []
for label in ['Res4 (453)', 'Res3 (1.9k)', 'Res2 (8k)', 'Full (35k)']:
    v, f = meshes[label]['vertices'], meshes[label]['faces']
    vc, fc, _ = cap_mesh(v, f)
    
    methods = {
        'Div.Theorem': volume_divergence_theorem(v, f),
        'Signed Tet.': volume_signed_tetrahedra(v, f),
        'Trimesh': abs(trimesh.Trimesh(vertices=v, faces=f).volume),
        'Div.Thm (capped)': volume_divergence_theorem(vc, fc),
        'Tet (capped)': volume_signed_tetrahedra(vc, fc),
    }
    
    # Slice-based (only for meshes with enough resolution)
    if len(v) >= 1000:
        vol_sl, _, _ = slice_based_volume(v, f, n_slices=80, smooth=True)
        methods['Slice-Based'] = vol_sl
        vol_tc_m, _, _ = volume_truncated_cone(v, f, n_slices=80)
        methods['Trunc. Cone'] = vol_tc_m
    
    for method, vol in methods.items():
        all_measurements.append({'Resolution': label, 'Method': method, 'Volume': vol})

df_all = pd.DataFrame(all_measurements)

# Pivot table
pivot = df_all.pivot_table(index='Resolution', columns='Method', values='Volume')
print(pivot.to_string(float_format='{:.8f}'.format))


def bland_altman(m1, m2):
    """Bland-Altman analysis: returns mean, diff, mean_diff, std_diff, LoA."""
    m1, m2 = np.asarray(m1), np.asarray(m2)
    mean_vals = (m1 + m2) / 2
    diffs = m1 - m2
    mean_diff = np.mean(diffs)
    std_diff = np.std(diffs, ddof=1)
    loa_upper = mean_diff + 1.96 * std_diff
    loa_lower = mean_diff - 1.96 * std_diff
    return mean_vals, diffs, mean_diff, std_diff, loa_upper, loa_lower


def compute_tem(diffs):
    """Technical Error of Measurement: TEM = sqrt(sum(d^2) / (2*n))"""
    diffs = np.asarray(diffs)
    return np.sqrt(np.sum(diffs**2) / (2 * len(diffs)))


def compute_cv(values):
    """Coefficient of Variation: CV = SD/mean * 100"""
    values = np.asarray(values)
    return np.std(values, ddof=1) / np.mean(values) * 100 if np.mean(values) != 0 else 0


def compute_icc(data_matrix):
    """ICC(3,1) - Two-way mixed, single measures, consistency.
    data_matrix: rows=subjects (resolutions), columns=methods"""
    n, k = data_matrix.shape
    grand_mean = data_matrix.mean()
    row_means = data_matrix.mean(axis=1)
    col_means = data_matrix.mean(axis=0)
    
    ss_total = np.sum((data_matrix - grand_mean)**2)
    ss_rows = k * np.sum((row_means - grand_mean)**2)
    ss_cols = n * np.sum((col_means - grand_mean)**2)
    ss_error = ss_total - ss_rows - ss_cols
    
    ms_rows = ss_rows / (n - 1)
    ms_error = ss_error / ((n - 1) * (k - 1))
    
    icc = (ms_rows - ms_error) / (ms_rows + (k - 1) * ms_error) if (ms_rows + (k - 1) * ms_error) != 0 else 0
    return icc

print('\nStatistical functions defined.')

In [ ]:
# Bland-Altman plots for key method pairs
# Use Full mesh measurements across different methods
full_only = df_all[df_all['Resolution'] == 'Full (35k)'].set_index('Method')['Volume']

# Get measurements across resolutions for paired comparisons
common_methods = ['Div.Theorem', 'Signed Tet.', 'Trimesh']
res_labels = ['Res4 (453)', 'Res3 (1.9k)', 'Res2 (8k)', 'Full (35k)']

pairs = [
    ('Div.Theorem', 'Signed Tet.'),
    ('Div.Theorem', 'Trimesh'),
    ('Div.Theorem', 'Div.Thm (capped)'),
]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (m1_name, m2_name) in enumerate(pairs):
    m1_vals, m2_vals = [], []
    for res in res_labels:
        subset = df_all[df_all['Resolution'] == res].set_index('Method')['Volume']
        if m1_name in subset.index and m2_name in subset.index:
            m1_vals.append(subset[m1_name])
            m2_vals.append(subset[m2_name])
    
    if len(m1_vals) < 2:
        axes[idx].text(0.5, 0.5, 'Insufficient data', ha='center', va='center', transform=axes[idx].transAxes)
        continue
    
    means, diffs, md, sd, upper, lower = bland_altman(m1_vals, m2_vals)
    
    ax = axes[idx]
    ax.scatter(means, diffs, s=80, color='steelblue', edgecolors='black', zorder=3)
    ax.axhline(md, color='red', linestyle='-', linewidth=1.5, label=f'Mean={md:.2e}')
    ax.axhline(upper, color='gray', linestyle='--', linewidth=1, label=f'+1.96SD={upper:.2e}')
    ax.axhline(lower, color='gray', linestyle='--', linewidth=1, label=f'-1.96SD={lower:.2e}')
    ax.fill_between(ax.get_xlim(), lower, upper, alpha=0.1, color='gray')
    ax.set_xlabel('Mean of Two Methods', fontsize=11)
    ax.set_ylabel('Difference', fontsize=11)
    ax.set_title(f'{m1_name} vs {m2_name}', fontsize=12, fontweight='bold')
    ax.legend(fontsize=8, loc='best')
    ax.grid(True, alpha=0.3)

plt.suptitle('Bland-Altman Analysis (Reference: 박도현 p.5)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Statistical Metrics: TEM, CV, ICC
print('=== Statistical Metrics (Reference: 박도현 p.7-8) ===\n')

# 1. TEM for each method pair
print('--- TEM (Technical Error of Measurement) ---')
print(f'{"Method Pair":>40} {"TEM":>14} {"rTEM (%)":>10}')
print('-' * 66)
for m1_name, m2_name in pairs:
    m1_vals, m2_vals = [], []
    for res in res_labels:
        subset = df_all[df_all['Resolution'] == res].set_index('Method')['Volume']
        if m1_name in subset.index and m2_name in subset.index:
            m1_vals.append(subset[m1_name])
            m2_vals.append(subset[m2_name])
    if len(m1_vals) >= 2:
        diffs = np.array(m1_vals) - np.array(m2_vals)
        tem = compute_tem(diffs)
        mean_val = np.mean(m1_vals + m2_vals)
        rtem = tem / mean_val * 100 if mean_val != 0 else 0
        print(f'{m1_name + " vs " + m2_name:>40} {tem:>14.2e} {rtem:>9.4f}%')

# 2. CV for each method across resolutions
print(f'\n--- CV (Coefficient of Variation) ---')
print(f'{"Method":>20} {"CV (%)":>10}')
print('-' * 32)
for method in common_methods + ['Div.Thm (capped)', 'Tet (capped)']:
    vals = df_all[df_all['Method'] == method]['Volume'].values
    if len(vals) >= 2:
        cv = compute_cv(vals)
        print(f'{method:>20} {cv:>9.4f}%')

# 3. ICC across methods
print(f'\n--- ICC (Intraclass Correlation Coefficient) ---')
# Build matrix: rows = resolutions, columns = methods (use common methods only)
icc_methods = ['Div.Theorem', 'Signed Tet.', 'Trimesh']
icc_matrix = []
for res in res_labels:
    row = []
    subset = df_all[df_all['Resolution'] == res].set_index('Method')['Volume']
    for m in icc_methods:
        if m in subset.index:
            row.append(subset[m])
    if len(row) == len(icc_methods):
        icc_matrix.append(row)

if len(icc_matrix) >= 2:
    icc_matrix = np.array(icc_matrix)
    icc_val = compute_icc(icc_matrix)
    print(f'  ICC(3,1) for {icc_methods}: {icc_val:.6f}')
    print(f'  Interpretation: {"Excellent" if icc_val > 0.9 else "Good" if icc_val > 0.75 else "Moderate" if icc_val > 0.5 else "Poor"}')

# ICC for capped methods
icc_methods_c = ['Div.Thm (capped)', 'Tet (capped)']
icc_matrix_c = []
for res in res_labels:
    row = []
    subset = df_all[df_all['Resolution'] == res].set_index('Method')['Volume']
    for m in icc_methods_c:
        if m in subset.index:
            row.append(subset[m])
    if len(row) == len(icc_methods_c):
        icc_matrix_c.append(row)

if len(icc_matrix_c) >= 2:
    icc_matrix_c = np.array(icc_matrix_c)
    icc_val_c = compute_icc(icc_matrix_c)
    print(f'  ICC(3,1) for {icc_methods_c}: {icc_val_c:.6f}')
    print(f'  Interpretation: {"Excellent" if icc_val_c > 0.9 else "Good" if icc_val_c > 0.75 else "Moderate" if icc_val_c > 0.5 else "Poor"}')

---
## Experiment 8: Comprehensive Comparison & Analysis

In [ ]:
# Final comprehensive comparison on Full mesh
print('=' * 90)
print('  COMPREHENSIVE VOLUME MEASUREMENT COMPARISON')
print('  Stanford Bunny - Full Resolution (35k vertices, 69k faces)')
print('=' * 90)

# Gather all methods
vol_slice, _, _ = slice_based_volume(vertices, faces, n_slices=100, smooth=True)
vol_tc, _, _ = volume_truncated_cone(vertices, faces, n_slices=100)

final_results = pd.DataFrame([
    {'Method': 'Divergence Theorem', 'Volume': vol_ref, 'Category': 'Surface Integral', 'Ref': '박문기'},
    {'Method': 'Signed Tetrahedra', 'Volume': volume_signed_tetrahedra(vertices, faces), 'Category': 'Geometric', 'Ref': '-'},
    {'Method': 'Trimesh (library)', 'Volume': abs(trimesh.Trimesh(vertices=vertices, faces=faces).volume), 'Category': 'Library', 'Ref': '-'},
    {'Method': 'Div.Thm (capped)', 'Volume': vol_capped_div, 'Category': 'Surface + Capping', 'Ref': '박문기 p.4'},
    {'Method': 'Tet (capped)', 'Volume': vol_capped_tet, 'Category': 'Geometric + Capping', 'Ref': '박문기 p.4'},
    {'Method': 'Slice-Based (100)', 'Volume': vol_slice, 'Category': 'Cross-Section', 'Ref': '박도현 p.7'},
    {'Method': 'Truncated Cone (100)', 'Volume': vol_tc, 'Category': 'Frustum Approx.', 'Ref': '박도현 p.5'},
    {'Method': 'Convex Hull', 'Volume': ConvexHull(vertices).volume, 'Category': 'Upper Bound', 'Ref': '-'},
])

# Use signed tetrahedra as reference (most robust for non-watertight)
ref_tet = volume_signed_tetrahedra(vertices, faces)
final_results['vs Tet (%)'] = abs(final_results['Volume'] - ref_tet) / ref_tet * 100

print(final_results.to_string(index=False, float_format='{:.8f}'.format))
print('=' * 90)

# Key findings
print('\n=== Key Findings ===')
print(f'1. Mesh is NOT watertight -> Divergence Theorem overestimates ({vol_ref:.6f} vs {ref_tet:.6f})')
print(f'2. Signed Tetrahedra and Trimesh agree closely (diff={abs(ref_tet - abs(trimesh.Trimesh(vertices=vertices, faces=faces).volume)):.2e})')
print(f'3. Capping changes volume by {abs(vol_capped_div/vol_ref - 1)*100:.2f}% (Div.Thm) / {abs(vol_capped_tet/ref_tet - 1)*100:.2f}% (Tet)')
print(f'4. Slice-based volume: {vol_slice:.6f} ({abs(vol_slice - ref_tet)/ref_tet*100:.2f}% vs Tet)')
print(f'5. Truncated cone: {vol_tc:.6f} ({abs(vol_tc - ref_tet)/ref_tet*100:.2f}% vs Tet)')
print(f'6. Icosphere Level 5 error: {df_ico.iloc[-1]["Vol Error (%)"]:.4f}% (on watertight mesh)')
hull_ratio = ConvexHull(vertices).volume / ref_tet
print(f'7. Convex Hull / Tet ratio: {hull_ratio:.2f}x (concavity indicator)')

In [ ]:
# Final comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. All methods comparison bar chart
ax = axes[0, 0]
colors_final = ['#2196F3', '#1976D2', '#0D47A1', '#FF5722', '#E64A19', '#4CAF50', '#388E3C', '#F44336']
bars = ax.barh(final_results['Method'], final_results['Volume'], color=colors_final, edgecolor='black', linewidth=0.5)
ax.axvline(x=ref_tet, color='red', linestyle='--', alpha=0.6, label=f'Tet ref={ref_tet:.6f}')
ax.set_xlabel('Volume', fontsize=12)
ax.set_title('All Volume Methods Comparison', fontsize=13, fontweight='bold')
ax.legend(fontsize=10); ax.grid(True, alpha=0.3, axis='x')

# 2. Icosphere convergence
ax = axes[0, 1]
ax.semilogy(df_ico['Level'], df_ico['Vol Error (%)'], 'bo-', markersize=10, linewidth=2)
for _, r in df_ico.iterrows():
    ax.annotate(f'{r["Vol Error (%)"]:.3f}%', (r['Level'], r['Vol Error (%)']),
                textcoords='offset points', xytext=(10, 5), fontsize=9)
ax.set_xlabel('Icosphere Level', fontsize=12)
ax.set_ylabel('Volume Error (%)', fontsize=12)
ax.set_title('Icosphere Precision (박문기 Table)', fontsize=13, fontweight='bold')
ax.set_xticks(range(1, 6)); ax.grid(True, alpha=0.3, which='both')

# 3. Slice count convergence
ax = axes[1, 0]
for smooth_label, marker, color in [('None', 'o', 'blue'), ('s=0', 's', 'green'), ('s=1e-6', '^', 'red')]:
    subset = df_slice[df_slice['Smoothing'] == smooth_label]
    ax.plot(subset['Slices'], subset['Volume'], f'{marker}-', color=color, 
            markersize=8, linewidth=1.5, label=f'Smooth={smooth_label}')
ax.plot(df_tc['Slices'], df_tc['Volume'], 'D--', color='purple', markersize=8, linewidth=1.5, label='Trunc. Cone')
ax.axhline(y=ref_tet, color='red', linestyle=':', alpha=0.5, label=f'Tet ref')
ax.set_xlabel('Number of Slices', fontsize=12)
ax.set_ylabel('Volume', fontsize=12)
ax.set_title('Slice Count Convergence (박도현 Approach)', fontsize=13, fontweight='bold')
ax.legend(fontsize=9, loc='best'); ax.grid(True, alpha=0.3)

# 4. Method error heatmap
ax = axes[1, 1]
# Create method x resolution error matrix
heat_methods = ['Div.Theorem', 'Signed Tet.', 'Trimesh', 'Div.Thm (capped)', 'Tet (capped)']
heat_data = []
for res in res_labels:
    row = []
    subset = df_all[df_all['Resolution'] == res].set_index('Method')['Volume']
    for m in heat_methods:
        if m in subset.index:
            row.append(subset[m])
        else:
            row.append(np.nan)
    heat_data.append(row)
heat_matrix = np.array(heat_data)
# Normalize by row mean
row_means = np.nanmean(heat_matrix, axis=1, keepdims=True)
heat_pct = (heat_matrix / row_means - 1) * 100

im = ax.imshow(heat_pct, cmap='RdBu_r', aspect='auto', vmin=-50, vmax=50)
ax.set_xticks(range(len(heat_methods)))
ax.set_xticklabels([m.replace(' ', '\n') for m in heat_methods], fontsize=8)
ax.set_yticks(range(len(res_labels)))
ax.set_yticklabels([r.split()[0] for r in res_labels], fontsize=10)
for i in range(len(res_labels)):
    for j in range(len(heat_methods)):
        if not np.isnan(heat_pct[i, j]):
            ax.text(j, i, f'{heat_pct[i,j]:+.1f}%', ha='center', va='center', fontsize=8)
ax.set_title('Method Deviation from Row Mean (%)', fontsize=13, fontweight='bold')
plt.colorbar(im, ax=ax, shrink=0.8)

plt.tight_layout()
plt.show()

In [ ]:
# Save all experiment results
experiment_output = {
    'experiment': 'Reference-Based Volumetric Measurements',
    'references': ['박문기 - 체적 측정', '박도현 - DIMA 69차'],
    'dataset': 'Stanford Bunny',
    'icosphere_convergence': {
        'target_volume': V_analytical,
        'results': df_ico[['Level', 'Vertices', 'Faces', 'Volume (Div)', 'Vol Error (%)']].to_dict('records'),
    },
    'bunny_volumes': {
        'divergence_theorem': float(vol_ref),
        'signed_tetrahedra': float(ref_tet),
        'trimesh': float(abs(trimesh.Trimesh(vertices=vertices, faces=faces).volume)),
        'capped_div': float(vol_capped_div),
        'capped_tet': float(vol_capped_tet),
        'slice_based_100': float(vol_slice),
        'truncated_cone_100': float(vol_tc),
        'convex_hull': float(ConvexHull(vertices).volume),
    },
    'capping': {
        'boundary_edges': len(boundary_edges),
        'cap_faces_added': int(n_cap),
    },
    'slice_convergence': df_slice.to_dict('records'),
    'truncated_cone_convergence': df_tc.to_dict('records'),
    'resolution_study': df_res[['Label', 'Vertices', 'Faces', 'Vol_Div', 'Vol_Tet', 'Area', 'Err_vs_Full (%)']].to_dict('records'),
}

# Save locally
local_path = os.path.join(WORK_DIR, 'experiment_results.json')
with open(local_path, 'w') as f:
    json.dump(experiment_output, f, indent=2, default=str)

# Copy to Drive if in Colab
if IN_COLAB and PROJECT_DIR != WORK_DIR:
    drive_output = os.path.join(PROJECT_DIR, 'experiment_results.json')
    shutil.copy(local_path, drive_output)
    print(f'Results saved to Drive: {drive_output}')
else:
    print(f'Results saved to: {local_path}')

print()
print('=' * 70)
print('  ALL EXPERIMENTS COMPLETE')
print('=' * 70)
print(f'  Exp 1: Icosphere L1-5 convergence (12.65% -> {df_ico.iloc[-1]["Vol Error (%)"]:.3f}%)')
print(f'  Exp 2: Green\'s Theorem cross-sections ({len(cross_sections)} slices computed)')
print(f'  Exp 3: Slice-based volume = {vol_slice:.8f}')
print(f'  Exp 4: Truncated cone volume = {vol_tc:.8f}')
print(f'  Exp 5: Capping added {n_cap} faces, WT={mesh_capped.is_watertight}')
print(f'  Exp 6: Resolution convergence (4 levels studied)')
print(f'  Exp 7: Bland-Altman + TEM/CV/ICC analysis')
print(f'  Exp 8: Comprehensive comparison (8 methods)')
print('=' * 70)